<a href="https://colab.research.google.com/github/EricAshby/EDA-Wine-Reviews/blob/main/TEDA1030_Mod5_project_EricAshby_08_30_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exploratory Data Analysis on Wine Reviews
Eric D. Ashby

##Introduction
This analysis aims to explore this data set of wines from various wineries and countries. Included in the data set is, among other things, information on the winery, country of origin, variety of wine, and description of the wine itself. Additional information is contained within the title of each review in text form.

The goal of this analysis is to answer the questions posed by the client.

##Purpose of Analysis
This project will make use of pandas data analysis tools including descriptive statistics, string functions, and regular expressions to accomplish its goal.

Specifically, this project will answer the following questions provided by the client:

1.   Which country produces wine with the most points, on average? (descriptive statistics)

2.   Which taster gives the lowest scores (points), on average?

3.   Which variety of wine is the most expensive, on average?

4.   Which year of wines has the best score (points), on average?

5.   Do reviews with the word "depth" in them tend to get better than average or worse than average points?

6.   Do reviews with the word "fruity" in them tend to get better than average or worse than average points?

7.   Do reviews with the word "herbal" in them tend to get better than average or worse than average points?

8.   Do reviews with more letters award more or less points, on average?

9.   Which region of the province Sicily & Sardinia produces the best wine, on average?

In [ ]:
import pandas as pd
df = pd.read_csv('wine.csv')

##Overview

Below is displayed the first five rows of the data set.  We can see that it includes 11 columns of data, each relating to characteristics of the wine, itself, the origin of the wine, the wine price, or the tasting of the wine.

In [ ]:
df.head()

,country,description,designation,points,price,province,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


Here, we have the metadata for the data set. We can see that, for those 11 columns, there are 129,971 rows. Some of the columns have a few holes in them, particularly designation and the taster's twitter handle. As neither of these pertains to our analysis goals, this will not be a problem.

There are also a few holes in country of origin, price, and province but, as the the number of missing entires is not overly large, we can likely assume out results to still be reasonbly representative of the whole.

Most important to our analysis is the description which, happily, is missing no entires. However, there remains a more significant hole in the tester name column, leaving us with a less reliable result for question 2 than we might have liked.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   country                129908 non-null  object 
 1   description            129971 non-null  object 
 2   designation            92506 non-null   object 
 3   points                 129971 non-null  int64  
 4   price                  120975 non-null  float64
 5   province               129908 non-null  object 
 6   taster_name            103727 non-null  object 
 7   taster_twitter_handle  98758 non-null   object 
 8   title                  129971 non-null  object 
 9   variety                129970 non-null  object 
 10  winery                 129971 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 10.9+ MB


Provided below are some descriptive statistics for the numeric datad in the data set, namely, points and price.

Note the right skew in the price data (i.e. the mean significantly exceeds the median) as well as the large standard deviation, suggesting that the price is extremely varied and that the price of some wines may be somewhat inflated.  This is particularly interesting as looking at the same statistics for points seems to suggest that the quality of wine (while still subjective) is fairly consistent and normally distributed.

In [ ]:
df.describe()

,points,price
count,129971.000000,120975.000000
mean,88.447138,35.363389
std,3.039730,41.022218
min,80.000000,4.000000
25%,86.000000,17.000000
50%,88.000000,25.000000
75%,91.000000,42.000000
max,100.000000,3300.000000


##Analysis


###1.   Which country produces wine with the most points, on average? (descriptive statistics)

The code below displays the average points scored for wines in the top 5 scoring countries. Note that the grouping method, `.groupby('country')`, excludes data with no country information. With this in mind, we can conclude that, for wines with country information provided, the country whose wine scores the most points on average is **England**.

In [ ]:
df[['country', 'points']].groupby('country').mean().sort_values(by = 'points', ascending = False).head()

,points
country,
England,91.581081
India,90.222222
Austria,90.101345
Germany,89.851732
Canada,89.369650


###2.   Which taster gives the lowest scores (points), on average?

Here, we have code that shows us the five tasters that give the lowest scores. As above, the `.groupby('taster_name')` method excludes entries with no data in the `tater_name` column. This leaves us with only 103,727 entries out of the 129,971 total (as can be seen in the overview). Nevertheless, we find that for this subset of data, the taster that gives the lowest scores on average is **Alexander Peartree**.

In [ ]:
df[['taster_name', 'points']].groupby('taster_name').mean().sort_values(by = 'points').head()

,points
taster_name,
Alexander Peartree,85.855422
Carrie Dykes,86.395683
Susan Kostrzewa,86.609217
Fiona Adams,86.888889
Michael Schachner,86.907493


###3.   Which variety of wine is the most expensive, on average?

Similar to previous two sections, this code provides us with the top 5 priced varieties on average. As can also be seen in the overview, the data set contains only one entry that lacks wine variety information. The result, then, that **Ramisco** is the highest priced wine on average, is almost precisely representative of the entire data set.

In [ ]:
df[['variety', 'price']].groupby('variety').mean().sort_values(by = 'price', ascending = False).head()

,price
variety,
Ramisco,495.000000
Terrantez,236.000000
Francisa,160.000000
Rosenmuskateller,150.000000
Malbec-Cabernet,113.333333


To sate curiousity, we find that our missing entry in the variety data is priced quite low compared to the highest priced wines.

In [ ]:
df[df['variety'].isna()]

,country,description,designation,points,price,province,taster_name,taster_twitter_handle,title,variety,winery
86909,Chile,"A chalky, dusty mouthfeel nicely balances this...",NaN,88,17.0,Maipo Valley,NaN,NaN,Carmen 1999 (Maipo Valley),NaN,Carmen


Looking back at the number of wines involved in the averages for the highest priced varieties, we find that there is only one entry for each of the top four.

In [ ]:
df[['variety', 'price']].groupby('variety').agg({'price' : ['mean', 'count']}).sort_values(by = ('price', 'mean'), ascending = False).head()

price      
                        mean count
variety                           
Ramisco           495.000000     1
Terrantez         236.000000     1
Francisa          160.000000     1
Rosenmuskateller  150.000000     1
Malbec-Cabernet   113.333333     3

Below, we have code that calculates the would-be average price for our top priced variety if the entry with missing variety data were actually of the same variety.  We find that, in this hypothetical case, Ramisco would still remain the highest priced variety at 256 with second still below at 236. Therefore, we can conclude with *certainty* that Ramisco is the highest priced variety of whine in the data set.

In [ ]:
highestPricedVariety = df[['variety', 'price']].groupby('variety').mean().sort_values(by = 'price', ascending = False).iloc[0,0]
missingPricedVariety = float(df[df['variety'].isna()]['price'])

(highestPricedVariety + missingPricedVariety) / 2

256.0

###4.   Which year of wines has the best score (points), on average?

The code below extracts the year from the text in the title column to a new column dubbed 'year'. The top 5 years in terms of average score are are then calculated and shown here. We see that the year with the highest score on average was in **1969**.

In [ ]:
#extract years to new column
yearRegex = '(\d{4})'
df['year'] = df['title'].str.extract(yearRegex)

#calculate average points by year and sort
df[['year', 'points']].groupby('year').mean().sort_values(by = 'points', ascending = False).head()

,points
year,
1969,98.0
1973,96.0
1952,95.5
1927,95.0
1945,95.0


###5.   Do reviews with the word "depth" in them tend to get better than average or worse than average points?

Here we have code that selects entries with reviews that mention 'depth' and calculates their average scores. The code then calculates the same for data set as a whole and compares the two. We find that wines with reviews containing the word 'depth' tend to get **better than average** scores.

In [ ]:
#select only entries with 'depth' in description
df['depth'] = df['description'].str.contains('depth')

#determine overall and depth-specific averages
depthAvg = df[df['depth']]['points'].mean()
overallAvg = df['points'].mean()
print("Average points for 'depth' wines:", depthAvg)
print("Overall average points:", overallAvg, "\n")

#determine if 'depth' reviews are associated w/ higher or lower than average scores
if depthAvg > overallAvg:
  print("Wines with reviews mentioning 'depth' tend to score higher than average.")
elif overallAvg > depthAvg:
  print("Wines with reviews mentioning 'depth' tend to score lower than average.")
else:
  print("Wines with reviews mentioning 'depth' tend to score niether above nor below average.")

Average points for 'depth' wines: 90.10941176470588
Overall average points: 88.44713820775404 

Wines with reviews mentioning 'depth' tend to score higher than average.


###6.   Do reviews with the word "fruity" in them tend to get better than average or worse than average points?

As above, this code determines whether "fruity" wines score higher or lower than average. We find that these wines tend to get **worse than average** scores.

In [ ]:
#select only entries with 'fruity' in description
df['fruity'] = df['description'].str.contains('fruity')

#determine overall and fruity-specific averages
fruityAvg = df[df['fruity']]['points'].mean()
overallAvg = df['points'].mean()
print("Average points for 'fruity' wines:", fruityAvg)
print("Overall average points:", overallAvg, "\n")

#determine if 'fruity' reviews are associated w/ higher or lower than average scores
if fruityAvg > overallAvg:
  print("Wines with reviews mentioning 'fruity' tend to score higher than average.")
elif overallAvg > fruityAvg:
  print("Wines with reviews mentioning 'fruity' tend to score lower than average.")
else:
  print("Wines with reviews mentioning 'fruity' tend to score niether above nor below average.")

Average points for 'fruity' wines: 87.61452145214521
Overall average points: 88.44713820775404 

Wines with reviews mentioning 'fruity' tend to score lower than average.


Interesting to note, however, this code shows that if we include wines with reviews containing the word "fruit", the result flips, though only just.

In [ ]:
#note: this code uses a slightly different method from above (out of preference) but to the same effect

#select only entries with 'fruit' in description
df_fruit = df[df['description'].str.contains('fruit')]

#determine overall and fruit-specific averages
fruitAvg = df_fruit['points'].mean()
overallAvg = df['points'].mean()
print("Average points for 'fruit' wines:", fruitAvg)
print("Overall average points:", overallAvg, "\n")

#determine if 'fruit' reviews are associated w/ higher or lower than average scores
if fruitAvg > overallAvg:
  print("Wines with reviews mentioning 'fruit' tend to score higher than average.")
elif overallAvg > fruitAvg:
  print("Wines with reviews mentioning 'fruit' tend to score lower than average.")
else:
  print("Wines with reviews mentioning 'fruit' tend to score niether above nor below average.")

Average points for 'fruit' wines: 88.583053498321
Overall average points: 88.44713820775404 

Wines with reviews mentioning 'fruit' tend to score higher than average.


###7.   Do reviews with the word "herbal" in them tend to get better than average or worse than average points?

Once again, the following code determines whether "herbal" wines score higher or lower on average.  We find that "herbal" wines tend to score **worse than average**.

In [ ]:
#select only entries with 'herbal' in description
df['herbal'] = df['description'].str.contains('herbal')

#determine overall and herbal-specific averages
herbalAvg = df[df['herbal']]['points'].mean()
overallAvg = df['points'].mean()
print("Average points for 'herbal' wines:", herbalAvg)
print("Overall average points:", overallAvg, "\n")

#determine if 'herb' reviews are associated w/ higher or lower than average scores
if herbalAvg > overallAvg:
  print("Wines with reviews mentioning 'herbal' tend to score higher than average.")
elif overallAvg > herbalAvg:
  print("Wines with reviews mentioning 'herbal' tend to score lower than average.")
else:
  print("Wines with reviews mentioning 'herbal' tend to score niether above nor below average.")

Average points for 'herbal' wines: 87.47001862197394
Overall average points: 88.44713820775404 

Wines with reviews mentioning 'herbal' tend to score lower than average.


And again, we find that the result flips (but only just) when we change the search to "herb".

In [ ]:
#note: this code uses a slightly different method from above (out of preference) but to the same effect

#select only entries with 'herb' in description
df_herb = df[df['description'].str.contains('herb')]

#determine overall and herb-specific averages
herbAvg = df_herb['points'].mean()
overallAvg = df['points'].mean()
print("Average points for 'herb' wines:", herbAvg)
print("Overall average points:", overallAvg, "\n")

#determine if 'herb' reviews are associated w/ higher or lower than average scores
if herbAvg > overallAvg:
  print("Wines with reviews mentioning 'herb' tend to score higher than average.")
elif overallAvg > herbAvg:
  print("Wines with reviews mentioning 'herb' tend to score lower than average.")
else:
  print("Wines with reviews mentioning 'herb' tend to score niether above nor below average.")

Average points for 'herb' wines: 88.51153987055807
Overall average points: 88.44713820775404 

Wines with reviews mentioning 'herb' tend to score higher than average.


###8.   Do reviews with more letters award more or less points, on average?

This code determines th correlation constant between the length of review and the points scored by a wine. We find that the review length and score have a moderate, positive correlation of ~0.558. This leads us to conclude that **longer reviews tend to award higher scores**.

In [ ]:
#create a new column for the number of characters in each review
df['review_length'] = df['description'].str.len()

#determine the correlation between review_length and points
df[['review_length', 'points']].corr()

,review_length,points
review_length,1.00000,0.55776
points,0.55776,1.00000


###9.   Which region of the province Sicily & Sardinia produces the best wine, on average?

Below, we have code that extracts region data from the title and creates a new column. The code then selects only the entries in the Province of Sicily & Sardinia to determine the top 5 highest scoring regions therein.  We see that **Faro** is the highest scoring region in Sicily & Sardinia.

In [ ]:
#create a regular expression to select for the region information contained in the title
regionRegex = '\((\w+)\)$'

#extract region data to new column for 'region'
df['region'] = df['title'].str.extract(regionRegex)

#create a filter for Sicily & Sardinia
SandSfilter = (df['province'] == 'Sicily & Sardinia')

#create filtered dataframe
df_SandS = df[SandSfilter]

#calculate average points by region
df_SandS[['region', 'points']].groupby('region').mean().sort_values(by = 'points', ascending = False).head()

,points
region,
Faro,94.000000
Alghero,91.500000
Eloro,90.200000
Salina,89.875000
Etna,89.869048


##Results

In this analysis, we find that England wines score highest on average, the taster Alexander Peartree gives the lowest scores on average, Ramisco has the highest average price by far, and 1969 was the highest scoring vintage.

We also find that wine with 'depth' generally scored better while 'fruity' and 'herbal' wines scored worse. However, changing the terms to 'fruit' and 'herb' respectively yields the opposite result for the latter two.

We see that tasters tend to give longer reviews for higher scoring wines.

Finally, we see that, on average, the province of Faro in Sicily & Sardinia produces the best wine as determined by score.